In [1]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict, Annotated
from langchain_core.messages import BaseMessage, HumanMessage
from langchain_groq import ChatGroq
from dotenv import load_dotenv

In [2]:
from langgraph.graph.message import add_messages

class ChatState(TypedDict):

    messages: Annotated[list[BaseMessage], add_messages]

In [3]:
load_dotenv()

True

In [4]:
llm = ChatGroq(
    model="openai/gpt-oss-120b",
    temperature=0.7
)


def chat_node(state: ChatState):

    # take user query from state
    messages = state['messages']

    # send to llm
    response = llm.invoke(messages)

    # response store state
    return {'messages': [response]}

In [5]:
graph = StateGraph(ChatState)

# add nodes
graph.add_node('chat_node', chat_node)

graph.add_edge(START, 'chat_node')
graph.add_edge('chat_node', END)

chatbot = graph.compile()

In [6]:
initial_state = {
    'messages': [HumanMessage(content='What is the capital of india')]
}

chatbot.invoke(initial_state)

{'messages': [HumanMessage(content='What is the capital of india', additional_kwargs={}, response_metadata={}, id='ff4eafcb-01e5-4e22-b93a-34faa4b39f48'),
  AIMessage(content='The capital of India is **New\u202fDelhi**.', additional_kwargs={'reasoning_content': 'The user asks: "What is the capital of india". Simple question. Answer: New Delhi. Provide brief answer.'}, response_metadata={'token_usage': {'completion_tokens': 45, 'prompt_tokens': 77, 'total_tokens': 122, 'completion_time': 0.094598692, 'completion_tokens_details': {'reasoning_tokens': 25}, 'prompt_time': 0.00301177, 'prompt_tokens_details': None, 'queue_time': 0.04933047, 'total_time': 0.097610462}, 'model_name': 'openai/gpt-oss-120b', 'system_fingerprint': 'fp_e10890e4b9', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019b732c-ff98-7a61-9586-787358e81e1c-0', usage_metadata={'input_tokens': 77, 'output_tokens': 45, 'total_tokens': 122, 'output_token_details'

In [7]:
while True:
  user_message = input("You: ")

  if user_message.lower() in ['exit', 'quit']:
      print("Exiting chat.")
      break
  response = chatbot.invoke({
      'messages': [HumanMessage(content=user_message)]})
  print(response)

{'messages': [HumanMessage(content='what are your capabilities', additional_kwargs={}, response_metadata={}, id='a78aeadb-b270-4b99-b913-98dbe77c471b'), AIMessage(content='I’m a versatile AI language model, and here’s a quick rundown of what I can do:\n\n### Core capabilities\n| Category | What I can help with |\n|----------|----------------------|\n| **Conversation** | Carry on casual chats, answer questions, provide explanations, brainstorm ideas, or just keep you company. |\n| **Information & Research** | Summarize articles, explain concepts, compare topics, give overviews of scientific, historical, technical, or cultural subjects. |\n| **Writing & Editing** | Draft essays, reports, stories, poems, emails, marketing copy, scripts, and more. I can proofread, suggest style improvements, and adapt tone to your audience. |\n| **Programming & Technical Tasks** | Write, debug, and explain code in many languages (Python, JavaScript, Java, C++, etc.), generate algorithms, create SQL queries